# Quantum ESPRESSO Workshop - Part 3: Wavefunction Cutoff Convergence

## Learning Objectives
1. Understand why ecutwfc convergence is essential
2. Perform systematic convergence tests
3. Determine the converged cutoff value
4. Understand the energy-accuracy tradeoff

---

## 1. Why Cutoff Convergence Matters

### The Plane-Wave Expansion

In QE, wavefunctions are expanded in plane waves:

$$\psi_n(\mathbf{r}) = \sum_{\mathbf{G}} c_{n,\mathbf{G}} e^{i(\mathbf{k}+\mathbf{G})\cdot\mathbf{r}}$$

where $\mathbf{G}$ are reciprocal lattice vectors.

### The Cutoff Criterion

We include only plane waves with kinetic energy below the cutoff:

$$\frac{\hbar^2}{2m}|\mathbf{k}+\mathbf{G}|^2 < E_{cut}$$

**ecutwfc** = $E_{cut}$ for wavefunctions (in Rydberg)

### Why It Matters

- **Too low**: Incomplete basis → inaccurate results
- **Too high**: Unnecessary computational cost
- **Goal**: Find the minimum cutoff that gives converged results

### Convergence Criterion

Total energy is converged when increasing ecutwfc changes the energy by less than a threshold (typically 1 meV/atom or 0.1 mRy/atom).

---

## 2. Setup

In [ ]:
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import re
import time
import json

# Paths
WORKSHOP_ROOT = Path('/home/claude/qe_workshop')
PSEUDO_DIR = WORKSHOP_ROOT / 'pseudopotentials'
OUTPUT_DIR = WORKSHOP_ROOT / 'outputs'

# Working directory for this notebook
WORK_DIR = OUTPUT_DIR / '03_ecutwfc_convergence'
WORK_DIR.mkdir(exist_ok=True)
(WORK_DIR / 'tmp').mkdir(exist_ok=True)

print(f"Working directory: {WORK_DIR}")

---

## 3. Helper Functions

In [ ]:
def generate_scf_input(prefix, ecutwfc, ecutrho, kpoints, pseudo_dir,
                       celldm1=10.26, conv_thr=1.0e-8):
    """
    Generate SCF input file for Silicon.
    """
    kx, ky, kz = kpoints
    
    input_text = f"""&CONTROL
    calculation = 'scf'
    prefix = '{prefix}'
    outdir = './tmp'
    pseudo_dir = '{pseudo_dir}'
    verbosity = 'high'
    tprnfor = .true.
    tstress = .true.
/

&SYSTEM
    ibrav = 2
    celldm(1) = {celldm1}
    nat = 2
    ntyp = 1
    ecutwfc = {ecutwfc}
    ecutrho = {ecutrho}
    occupations = 'smearing'
    smearing = 'cold'
    degauss = 0.01
/

&ELECTRONS
    conv_thr = {conv_thr}
    mixing_beta = 0.7
/

ATOMIC_SPECIES
    Si  28.0855  Si.upf

ATOMIC_POSITIONS {{crystal}}
    Si  0.00  0.00  0.00
    Si  0.25  0.25  0.25

K_POINTS {{automatic}}
    {kx} {ky} {kz} 0 0 0
"""
    return input_text


def parse_total_energy(output_text):
    """
    Extract total energy from pw.x output.
    Returns energy in Ry or None if not found.
    """
    for line in output_text.split('\n'):
        if '!' in line and 'total energy' in line:
            match = re.search(r'=\s+([\d.E+-]+)\s+Ry', line)
            if match:
                return float(match.group(1))
    return None


def parse_scf_converged(output_text):
    """Check if SCF converged."""
    return 'convergence has been achieved' in output_text


def run_pwscf(input_file, timeout=300):
    """
    Run pw.x and return (output_text, elapsed_time, converged).
    """
    input_file = Path(input_file)
    output_file = input_file.with_suffix('.out')
    work_dir = input_file.parent
    
    start = time.time()
    result = subprocess.run(
        ['pw.x', '-in', input_file.name],
        capture_output=True,
        text=True,
        cwd=work_dir,
        timeout=timeout
    )
    elapsed = time.time() - start
    
    with open(output_file, 'w') as f:
        f.write(result.stdout)
    
    converged = parse_scf_converged(result.stdout)
    
    return result.stdout, elapsed, converged

---

## 4. Ecutwfc Convergence Test

We'll test ecutwfc values from 20 to 80 Ry in steps of 5 Ry.

**Important**: For this test, we fix:
- K-points: 6×6×6 (should be adequate for Si)
- ecutrho = 8 × ecutwfc (appropriate for PAW pseudopotentials)
- Lattice parameter: experimental value (10.26 Bohr)

In [ ]:
# Define test parameters
ecutwfc_values = np.arange(20, 85, 5)  # 20, 25, 30, ..., 80 Ry
ecutrho_factor = 8  # ecutrho = 8 * ecutwfc for PAW
kpoints = (6, 6, 6)  # Fixed k-point grid

print("Ecutwfc Convergence Test Parameters")
print("=" * 50)
print(f"ecutwfc range: {ecutwfc_values[0]} - {ecutwfc_values[-1]} Ry")
print(f"Number of tests: {len(ecutwfc_values)}")
print(f"ecutrho = {ecutrho_factor} × ecutwfc")
print(f"K-points: {kpoints}")
print(f"Lattice parameter: 10.26 Bohr (experimental)")

In [ ]:
# Run convergence tests
results = []

print("\nRunning ecutwfc convergence tests...")
print("=" * 70)
print(f"{'ecutwfc (Ry)':<15} {'ecutrho (Ry)':<15} {'Energy (Ry)':<20} {'Time (s)':<10} {'Status'}")
print("-" * 70)

for ecutwfc in ecutwfc_values:
    ecutrho = ecutwfc * ecutrho_factor
    prefix = f'si_ecut{int(ecutwfc)}'
    
    # Generate input
    input_text = generate_scf_input(
        prefix=prefix,
        ecutwfc=ecutwfc,
        ecutrho=ecutrho,
        kpoints=kpoints,
        pseudo_dir=PSEUDO_DIR,
        conv_thr=1.0e-8
    )
    
    # Write input file
    input_file = WORK_DIR / f'{prefix}.in'
    with open(input_file, 'w') as f:
        f.write(input_text)
    
    # Run calculation
    output, elapsed, converged = run_pwscf(input_file)
    energy = parse_total_energy(output)
    
    status = '✓' if converged else '✗'
    energy_str = f'{energy:.8f}' if energy else 'N/A'
    
    print(f"{ecutwfc:<15.1f} {ecutrho:<15.1f} {energy_str:<20} {elapsed:<10.2f} {status}")
    
    results.append({
        'ecutwfc': ecutwfc,
        'ecutrho': ecutrho,
        'energy_ry': energy,
        'time_s': elapsed,
        'converged': converged
    })

print("=" * 70)
print("Convergence tests complete!")

In [ ]:
# Save results to JSON for later use
results_file = WORK_DIR / 'ecutwfc_convergence_results.json'
with open(results_file, 'w') as f:
    json.dump(results, f, indent=2)
print(f"Results saved to: {results_file}")

---

## 5. Analyze Convergence

In [ ]:
# Extract converged results
converged_results = [r for r in results if r['converged'] and r['energy_ry'] is not None]

if len(converged_results) < 2:
    print("ERROR: Not enough converged calculations for analysis!")
else:
    ecutwfc_arr = np.array([r['ecutwfc'] for r in converged_results])
    energy_arr = np.array([r['energy_ry'] for r in converged_results])
    time_arr = np.array([r['time_s'] for r in converged_results])
    
    # Reference energy: highest cutoff
    E_ref = energy_arr[-1]
    delta_E = (energy_arr - E_ref) * 1000  # Convert to mRy
    delta_E_meV = (energy_arr - E_ref) * 13605.693 / 2  # meV per atom (2 atoms)
    
    print("Convergence Analysis")
    print("=" * 80)
    print(f"Reference: ecutwfc = {ecutwfc_arr[-1]} Ry, E = {E_ref:.8f} Ry")
    print("\nNote: ΔE is relative to the highest cutoff (should approach zero)")
    print(f"\n{'ecutwfc (Ry)':<15} {'Energy (Ry)':<18} {'ΔE (mRy)':<12} {'ΔE (meV/atom)':<15} {'Time (s)'}")
    print("-" * 80)
    
    for i in range(len(converged_results)):
        print(f"{ecutwfc_arr[i]:<15.1f} {energy_arr[i]:<18.8f} {delta_E[i]:<12.4f} {delta_E_meV[i]:<15.4f} {time_arr[i]:.2f}")
    
    print("=" * 80)

In [ ]:
# Determine converged cutoff
# Criterion: ΔE < 1 meV/atom
convergence_threshold_meV = 1.0  # meV per atom

print(f"\nConvergence criterion: |ΔE| < {convergence_threshold_meV} meV/atom")
print("-" * 50)

converged_cutoff = None
for i in range(len(converged_results) - 1):
    if abs(delta_E_meV[i]) < convergence_threshold_meV:
        converged_cutoff = ecutwfc_arr[i]
        print(f"\n✓ Converged cutoff: {converged_cutoff} Ry")
        print(f"  At this cutoff: ΔE = {delta_E_meV[i]:.4f} meV/atom")
        break

if converged_cutoff is None:
    print("\n⚠ No cutoff satisfies the convergence criterion!")
    print("  Consider testing higher cutoff values.")
else:
    # Recommendation with safety margin
    recommended_cutoff = converged_cutoff + 5  # Add 5 Ry margin
    print(f"\n★ Recommended cutoff (with 5 Ry safety margin): {recommended_cutoff} Ry")

---

## 6. Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Absolute energy vs cutoff
ax1 = axes[0, 0]
ax1.plot(ecutwfc_arr, energy_arr, 'bo-', markersize=8, linewidth=2)
ax1.set_xlabel('ecutwfc (Ry)', fontsize=12)
ax1.set_ylabel('Total Energy (Ry)', fontsize=12)
ax1.set_title('Total Energy vs Wavefunction Cutoff', fontsize=14)
ax1.grid(True, alpha=0.3)

# Plot 2: Energy difference (convergence)
ax2 = axes[0, 1]
ax2.semilogy(ecutwfc_arr[:-1], np.abs(delta_E_meV[:-1]), 'ro-', markersize=8, linewidth=2)
ax2.axhline(y=convergence_threshold_meV, color='g', linestyle='--', 
            label=f'Threshold = {convergence_threshold_meV} meV/atom')
if converged_cutoff:
    ax2.axvline(x=converged_cutoff, color='b', linestyle=':', alpha=0.7,
                label=f'Converged at {converged_cutoff} Ry')
ax2.set_xlabel('ecutwfc (Ry)', fontsize=12)
ax2.set_ylabel('|ΔE| (meV/atom)', fontsize=12)
ax2.set_title('Energy Convergence', fontsize=14)
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

# Plot 3: Computational time vs cutoff
ax3 = axes[1, 0]
ax3.plot(ecutwfc_arr, time_arr, 'gs-', markersize=8, linewidth=2)
ax3.set_xlabel('ecutwfc (Ry)', fontsize=12)
ax3.set_ylabel('Wall Time (s)', fontsize=12)
ax3.set_title('Computational Cost vs Cutoff', fontsize=14)
ax3.grid(True, alpha=0.3)

# Plot 4: Energy difference (linear scale, zoomed)
ax4 = axes[1, 1]
ax4.plot(ecutwfc_arr, delta_E_meV, 'mo-', markersize=8, linewidth=2)
ax4.axhline(y=0, color='k', linestyle='-', alpha=0.3)
ax4.axhline(y=convergence_threshold_meV, color='g', linestyle='--', alpha=0.7)
ax4.axhline(y=-convergence_threshold_meV, color='g', linestyle='--', alpha=0.7)
ax4.fill_between(ecutwfc_arr, -convergence_threshold_meV, convergence_threshold_meV, 
                 color='green', alpha=0.1, label='Converged region')
ax4.set_xlabel('ecutwfc (Ry)', fontsize=12)
ax4.set_ylabel('ΔE (meV/atom)', fontsize=12)
ax4.set_title('Energy Difference (Linear Scale)', fontsize=14)
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(str(WORK_DIR / 'ecutwfc_convergence.png'), dpi=150, bbox_inches='tight')
plt.show()
print(f"\nFigure saved to: {WORK_DIR / 'ecutwfc_convergence.png'}")

---

## 7. Understanding the Results

### Key Observations

1. **Total energy decreases** as cutoff increases (more complete basis)
2. **Convergence is variational**: True energy is approached from above
3. **Diminishing returns**: Energy change becomes smaller at high cutoffs
4. **Cost increases**: Higher cutoff = more plane waves = more computation

### Practical Guidelines

| Accuracy Level | Convergence Criterion | Typical Use |
|----------------|----------------------|-------------|
| Low | 10 meV/atom | Quick tests, screening |
| Medium | 1 meV/atom | Standard calculations |
| High | 0.1 meV/atom | Accurate energetics |
| Ultra-high | 0.01 meV/atom | Phonons, response functions |

In [ ]:
# Summary table with different convergence criteria
criteria = [10.0, 1.0, 0.1]

print("\nRecommended Cutoffs for Different Accuracy Levels")
print("=" * 60)
print(f"{'Criterion (meV/atom)':<25} {'Converged ecutwfc (Ry)':<20} {'Time (s)'}")
print("-" * 60)

for criterion in criteria:
    for i in range(len(converged_results) - 1):
        if abs(delta_E_meV[i]) < criterion:
            print(f"{criterion:<25.1f} {ecutwfc_arr[i]:<20.0f} {time_arr[i]:.2f}")
            break
    else:
        print(f"{criterion:<25.1f} {'> ' + str(int(ecutwfc_arr[-1])):<20} N/A")

print("=" * 60)

---

## 8. Store Converged Value for Next Notebooks

In [ ]:
# Save the converged parameters for use in subsequent notebooks
converged_params = {
    'ecutwfc': float(converged_cutoff) if converged_cutoff else 40.0,
    'ecutwfc_recommended': float(converged_cutoff + 5) if converged_cutoff else 45.0,
    'ecutrho_factor': ecutrho_factor,
    'convergence_criterion_meV': convergence_threshold_meV,
    'reference_energy_ry': float(E_ref),
    'kpoints_used': list(kpoints)
}

params_file = WORKSHOP_ROOT / 'converged_parameters.json'
with open(params_file, 'w') as f:
    json.dump(converged_params, f, indent=2)

print("Converged parameters saved for subsequent notebooks:")
print(json.dumps(converged_params, indent=2))
print(f"\nSaved to: {params_file}")

---

## Summary

In this notebook, we have:

1. ✓ Understood why ecutwfc convergence is essential
2. ✓ Performed systematic convergence tests (20-80 Ry)
3. ✓ Determined the converged cutoff value
4. ✓ Visualized the convergence behavior
5. ✓ Saved converged parameters for later use

### Key Findings

For Silicon with PAW pseudopotential (PBE):
- **Converged ecutwfc** (1 meV/atom): ~{converged_cutoff} Ry (if determined)
- **Recommended ecutwfc** (with safety margin): ~{converged_cutoff + 5 if converged_cutoff else 45} Ry
- **ecutrho**: 8 × ecutwfc for PAW pseudopotentials

### Important Notes

1. **Convergence is system-specific**: Different elements/pseudopotentials require different cutoffs
2. **Always test convergence** when starting a new project
3. **Keep records**: Document your convergence tests

### Next Notebook
→ **04_Kpoint_Convergence.ipynb**: K-point sampling convergence testing